In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'w': 0, ' ': 1, 'd': 2, 'h': 3, 'r': 4, 'o': 5, 'e': 6, 'l': 7},
 {0: 'w', 1: ' ', 2: 'd', 3: 'h', 4: 'r', 5: 'o', 6: 'e', 7: 'l'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(embedding_size, hidden_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(hidden_size, output_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh + h_prev @ Whh + bh)
    probs = softmax(h_next @ Why + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([3, 6, 7, 7, 5, 1, 0, 5, 4, 7], [6, 7, 7, 5, 1, 0, 5, 4, 7, 2])

In [9]:
sample("h", 4)

'hwor'

In [26]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # 2nd layer
        dL_Why += dL_dz2 * hs[t].T
        dL_dby += dL_dz2

        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = (dL_dz2 @ Why.T) + dF_dh 

        dL_dWxh += dL_dh * dh_dz1 * xs[t].T
        dL_dWhh += dL_dh * dh_dz1 * hs[t].T
        dL_dbh += dL_dh * dh_dz1

        dF_dh = dL_dh * (dh_dz1 @ Whh.T)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 5.122753882063285
[Loss]: 11.095246920650402
[Loss]: 8.554840069864063
[Loss]: 4.226851572140599
[Loss]: 6.282191211155975
[Loss]: 19.620474084043614
[Loss]: 37.057248433580725
[Loss]: 28.842950577724288
[Loss]: 0.3405652928045862
[Loss]: 0.45549371931877686
[Loss]: 7.339589587308039
[Loss]: 11.553390264808915
[Loss]: 6.685820575268527
[Loss]: 2.0277017363159366
[Loss]: 4.464625594727593
[Loss]: 5.538300242719268
[Loss]: 8.54128007689768
[Loss]: 11.7598644598439
[Loss]: 0.6400021888355423
[Loss]: 10.208360505806802
[Loss]: 1.162722630227682
[Loss]: 0.23649887117646465
[Loss]: 0.26332940426910817
[Loss]: 0.25785018465312637
[Loss]: 0.4956125698750249
[Loss]: 3.6033541483492537
[Loss]: 4.8530544888802245
[Loss]: 4.23885081102703
[Loss]: 5.19579297376276
[Loss]: 9.993900973867717
[Loss]: 0.634078454279546
[Loss]: 0.06591927694338401
[Loss]: 0.06543873268919828
[Loss]: 0.3192845189321185
[Loss]: 3.46441430662075
[Loss]: 0.6070381489733823
[Loss]: 2.8859862767660425
[Loss]: 1.597757

In [27]:
sample("h", 10)

'ello world'